In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import operator
import math
import random

from scipy.linalg import norm
from scipy.spatial.distance import cdist
from collections import defaultdict

from sklearn import datasets
from sklearn.datasets import make_blobs

In [148]:
# ignoring warnings
import warnings
warnings.simplefilter("ignore")
np.random.seed(42)

In [149]:
#Load Dataset
Home_data = pd.read_csv('HomeC.csv')
nan_removed = pd.DataFrame.dropna(pd.DataFrame(Home_data))
nan_removed.value_counts()

time        use [kW]  gen [kW]  House overall [kW]  Dishwasher [kW]  Furnace 1 [kW]  Furnace 2 [kW]  Home office [kW]  Fridge [kW]  Wine cellar [kW]  Garage door [kW]  Kitchen 12 [kW]  Kitchen 14 [kW]  Kitchen 38 [kW]  Barn [kW]  Well [kW]  Microwave [kW]  Living room [kW]  Solar [kW]  temperature  icon         humidity  visibility  summary     apparentTemperature  pressure  windSpeed  cloudCover  windBearing  precipIntensity  dewPoint  precipProbability
1451624400  0.932833  0.003483  0.932833            0.000033         0.020700        0.061917        0.442633          0.124150     0.006983          0.013083          0.000417         0.000150         0.000000         0.031350   0.001017   0.004067        0.001517          0.003483    36.14        clear-night  0.62      10.00       Clear       29.26                1016.91   9.18       cloudCover  282.0        0.0000           24.40     0.00                 1
1451960350  0.498450  0.054750  0.498450            0.000000         0.020100

In [150]:
duplicate_removed = (nan_removed[['use [kW]','summary', 'windSpeed', 'apparentTemperature', 'windBearing', 'temperature']].drop_duplicates()).round()
duplicate_removed['summary'].value_counts()

Clear                       374034
Partly Cloudy                61827
Light Rain                   27164
Drizzle                      10304
Overcast                      5991
Rain                          5136
Mostly Cloudy                 4517
Light Snow                    4309
Flurries                      1785
Breezy                        1543
Snow                          1140
Breezy and Partly Cloudy      1027
Foggy                          972
Rain and Breezy                174
Heavy Snow                     169
Flurries and Breezy            115
Breezy and Mostly Cloudy        58
Dry                             50
Name: summary, dtype: int64

In [151]:
# df = pd.DataFrame(duplicate_removed)
df_selected = duplicate_removed[duplicate_removed['summary'].str.contains('Light Snow', case = False, regex = True).fillna(False) | duplicate_removed['summary'].str.contains('Heavy Snow', case = False, regex = True).fillna(False) | duplicate_removed['summary'].str.contains('Foggy', case = False).fillna(False)] 
X = (df_selected[['windSpeed', 'apparentTemperature', 'windBearing', 'temperature']]).values
y = df_selected['use [kW]'].values
data = X

In [152]:

# Mendefinisikan fungsi jarak euclidean
def euclidean_distance(point1,point2):
    dis=0
    for i in range(len(point1)):
        dis+=(point1[i]-point2[i])**2
    return dis**0.5

# Initializing the Membership Matrix
def initialize_membership_matrix(n,c):
    member_mat=list()
    for i in range(n):
        random_list=[random.random() for x in range(c)]
        summation=sum(random_list)
        for i in range(len(random_list)):
            random_list[i]=random_list[i]/summation
        member_mat.append(random_list)
    return member_mat

def update_centroid(member_mat):
    centroids={}
    for j in range(c):
        temp=[]
        for k in range(p):      
            add=0
            for i in range(n):
                add+=member_mat[i][j]**m
            x=0
            for i in range(n):
                x+=(member_mat[i][j]**m)*(data[i][k])
            val=x/add
            temp.append(val)
        centroids[j]=temp
    return centroids

def update_membership_matrix(member_mat,centroids):
    ratio=float(2/(m-1))
    for i in range(n):
        distances=list()
        for j in range(c):
            distances.append(euclidean_distance(data[i],centroids[j]))
        for j in range(c):
            den = sum([math.pow(float(distances[j]/distances[q]), ratio) for q in range(c)])
            member_mat[i][j] = float(1/den)         
            
    return member_mat
        
def find_cluster(member_mat):
    clusters=list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(member_mat[i]))
        clusters.append(idx)
    return clusters

def check(old_member_mat,member_mat):
    diff=0
    for i in range(n):
        for j in range(c):
            diff+=old_member_mat[i][j]-member_mat[i][j]
    if(diff<0.01):
        return True
    return False

def fuzzy_c_mean(max_iter):
    member_mat=initialize_membership_matrix(n,c)
    for i in range(max_iter):
        centroids=update_centroid(member_mat)
        old_member_mat=member_mat
        member_mat=update_membership_matrix(member_mat,centroids)
        cluster=find_cluster(member_mat)
        if(check(old_member_mat,member_mat))<0.01:
            print(i)
            break    
    
    return cluster,centroids

# labeling the clusters
def label_clusters(clusters):
    z=0
    o=0
    t=0
    dict=defaultdict(int)

    for i in range(50):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    classes=[]
    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(50,100):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2

    fin1=max(z,max(o,t))
    
    classes.append(dict[fin1])
    z=0
    o=0
    t=0
    for i in range(100,150):
        if(clusters[i]==0):
            z=z+1
        elif(clusters[i]==1):
            o=o+1
        else:
            t=t+1
    dict[z]=0
    dict[o]=1
    dict[t]=2
    fin1=max(z,max(o,t))
    classes.append(dict[fin1])
    
    return classes

def confusion_matrix(clusters,classes,labels):
    c_matrix=[[0 for i in range(labels)] for j in range(labels)]
    m=0
    n=0
    q=0
    for i in range(50):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[0][0]=m
    c_matrix[0][1]=n
    c_matrix[0][2]=q
    m=0
    n=0
    q=0
    for i in range(50,100):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[1][0]=m
    c_matrix[1][1]=n
    c_matrix[1][2]=q
    m=0
    n=0
    q=0

    for i in range(100,150):
        if clusters[i]==classes[0]:
            m=m+1
        elif clusters[i]==classes[1]:
            n=n+1
        else:
            q=q+1
    c_matrix[2][0]=m
    c_matrix[2][1]=n
    c_matrix[2][2]=q
    
    return c_matrix

# Performance Metric
class Metrics:
    def __init__(self,confusion_m):
        self.confusion_m=confusion_m
        self.total=np.sum(confusion_m)
        self.diagonal=np.sum(np.diag(confusion_m))
    
    def accuracy(self):
        accuracy=(self.diagonal/self.total)
        return accuracy
    
    def recall(self):
        recall=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=1)
        recall=np.mean(recall)
        return recall
    
    def precision(self):
        precision=np.diag(self.confusion_m)/np.sum(self.confusion_m,axis=0)
        precision=np.mean(precision)
        return precision
    
    def f1_score(self,precision,recall):
        f1_score=(2*precision*recall)/(precision+recall)
        return f1_score

In [153]:
# utility values
m = 2
n = len(data)
c = 3
p = len(data[0])
max_iter = 100

# Prediksi menggunakan FCM Clustering
clusters,centroids=fuzzy_c_mean(max_iter)
print("Final Centroid points are:")
print(centroids)

classes=label_clusters(clusters)

Final Centroid points are:
{0: [2.977815938122485, 27.316160533407714, 170.73999214553024, 29.106432219559608], 1: [6.0000363790704885, 19.042820166762542, 31.955766249311118, 25.007996497816745], 2: [5.168998402723652, 27.89697731237377, 333.85639745219794, 32.28480673231899]}


In [154]:
# Performance
matrix=confusion_matrix(clusters,classes,3)
performance=Metrics(matrix)

accuracy=performance.accuracy()
recall=performance.recall()
precision=performance.precision()
f1_score=performance.f1_score(precision,recall)

print('confusion matrix is:',end='\n')
print(np.array(matrix),end='\n')

print("Accuracy of the model is {}".format(accuracy*100))
print("Recall of the model is {}".format(recall*100))
print("Precision of the model is {}".format(precision*100))
print("F1-Score of the model is {}".format(f1_score*100))

confusion matrix is:
[[50  0  0]
 [50  0  0]
 [16  0 34]]
Accuracy of the model is 56.00000000000001
Recall of the model is 56.00000000000001
Precision of the model is nan
F1-Score of the model is nan
